In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout, RandomRotation
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16, resnet
from tensorflow.python.keras.optimizer_v2 import adam

In [2]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [3]:
def load_data():
    image_path = "./dataset/images.npy"
    emotion_path = "./dataset/emotions_multi.npy"
    
    images = np.load(image_path)
    images = tf.convert_to_tensor(images)
    images = layers.Rescaling(1./127.5, offset= -1)(images)
    images = tf.image.grayscale_to_rgb(images)
    emotions = np.load(emotion_path)
    emotions = tf.convert_to_tensor(emotions)
    
    training_samples = 28317
    validation_samples = 3541
    training_size = training_samples + validation_samples
    
    training_images = images[:training_size]
    test_images = images[training_size:]
    training_emotions = emotions[:training_size]
    test_emotions = emotions[training_size:]
    return training_images, training_emotions, test_images, test_emotions
training_images, training_emotions, test_images, test_emotions = load_data()

In [4]:
training_images_flipped = tf.image.flip_left_right(training_images)
training_images_rr_ccw = RandomRotation((-1/12,-1/36), fill_mode='constant')(training_images)
training_images_rr_cw = RandomRotation((1/36,1/12), fill_mode='constant')(training_images)

In [5]:
training_images_augmented = tf.concat([training_images,training_images_flipped,training_images_rr_ccw,training_images_rr_cw], 0)
training_emotions_augmented = tf.concat([training_emotions,training_emotions,training_emotions,training_emotions], 0)

In [6]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [7]:
def train(model, learning_rate, loss, num_epochs, batch_size):
    model.compile(optimizer=adam.Adam(learning_rate=learning_rate), 
                  loss=loss, 
                  metrics = [model_acc])
    return model.fit(x=training_images_augmented,
                     y=training_emotions_augmented,
                     batch_size=batch_size,
                     epochs=num_epochs,
                     validation_data=(test_images, test_emotions))

In [ ]:
if not os.path.isdir('./results/'):
    os.mkdir('./results/')

learning_rate = 1e-4
num_epochs = 40
loss = losses.MeanSquaredError()
batch_size = 32
val_accs = []

base_model = vgg16.VGG16(include_top=False, weights="imagenet", input_shape=(48,48,3))
base_model.trainable=True
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(emotions_count, activation='softmax'),
])

history = train(model, learning_rate, loss, num_epochs, batch_size)
history_save_path = './results/random-rotation.txt'
with open(history_save_path, 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

/Users/duanruxiao/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4526: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/40
   1/3983 [..............................] - ETA: 4:28:38 - loss: 0.0684 - model_acc: 0.1875